## GSKY WPS Polygon Drill using python 

### GetCapabilities

In [1]:
# Import python libraries


#######################################################
#   OWSLib is a Python package for client             #
#   programming with Open Geospatial Consortium (OGC) #
#   web service (hence OWS)interface standards,       # 
#   and their related content models.                 # 
#######################################################


from owslib.wcs import WebCoverageService
from owslib.wps import WebProcessingService
from owslib.wps import printInputOutput
from PIL import Image
%matplotlib inline

### Link to GSKY WPS

gsky_url = 'http://130.56.242.16/ows/internal/wps'
#gsky_url = 'http://130.56.242.16/ows/edge/generic_wps'

In [2]:
# Define the WPS

wps = WebProcessingService(gsky_url, verbose=False, skip_caps=True)
wps.getcapabilities()
wps.identification.title

'GSKY WPS'

In [3]:
# View the wps abstract

print(wps.identification.abstract)
print(wps.identification.type)
print(wps.identification.accessconstraints)
print(wps.identification.version)

GSKY - A Scalable, Distributed Geospatial Data Service. https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/dc9fb2db-8d6f-4b76-a734-93ac7fbc9201
WPS
None
1.0.0


In [4]:
# Print WPS operations 

for operation in wps.operations:
    print(operation.name)

GetCapabilities
DescribeProcess
Execute


In [5]:
# List the WPS processes available

for process in wps.processes:
    print(process.identifier)

LS8 NBART Polygon Drill Raw Bands
LS8 NBART Pixel Drill Raw Bands
LS8 NBART Polygon Drill NDVI NDWI
LS8 NBART Pixel Drill NDVI NDWI


In [6]:
# List the WPS processes available

for process in wps.processes:
    print(process.identifier, process.title)

LS8 NBART Polygon Drill Raw Bands LS8 NBART Polygon Drill (Raw bands)
LS8 NBART Pixel Drill Raw Bands LS8 NBART Pixel Drill (Raw bands)
LS8 NBART Polygon Drill NDVI NDWI LS8 NBART Polygon Drill (NDVI NDWI)
LS8 NBART Pixel Drill NDVI NDWI LS8 NBART Pixel Drill (NDVI NDWI)


### DescribeProcess

Determine how a specific process needs to be invoked - i.e. what are its input parameters, and output result:

In [7]:
wps = WebProcessingService(gsky_url, verbose=False, skip_caps=True)
process = wps.describeprocess('LS8 NBART Polygon Drill Raw Bands')

In [8]:
process.identifier

'LS8 NBART Polygon Drill Raw Bands'

In [9]:
for output in process.processOutputs:
    print(output.title, output.identifier, output.dataType, output.defaultValue)   

Time Series Output Result ComplexData <owslib.wps.ComplexData object at 0x122a76860>


In [10]:
for input in process.dataInputs:
    printInputOutput(input)

 identifier=start_datetime, title=Start datetime, abstract=None, data type=ComplexData
 Supported Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://www.w3.org/TR/xmlschema-2/#dateTime
 Default Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://www.w3.org/TR/xmlschema-2/#dateTime 
 minOccurs=0, maxOccurs=1
 identifier=end_datetime, title=End datetime, abstract=None, data type=ComplexData
 Supported Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://www.w3.org/TR/xmlschema-2/#dateTime
 Default Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://www.w3.org/TR/xmlschema-2/#dateTime 
 minOccurs=0, maxOccurs=1
 identifier=geometry, title=Polygon geometry, abstract=None, data type=ComplexData
 Supported Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://geojson.org/geojson-spec.html#polygon
 Default Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://geojson.org/geojson-spec.

In [11]:
for output in process.processOutputs:
    printInputOutput(input)

 identifier=geometry, title=Polygon geometry, abstract=None, data type=ComplexData
 Supported Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://geojson.org/geojson-spec.html#polygon
 Default Value: mimeType=application/vnd.geo+json, encoding=None, schema=http://geojson.org/geojson-spec.html#polygon 
 minOccurs=1, maxOccurs=1


### Execute

Submit an execute request (extraction of a climate index variable over a specific GML polygon, for a given period of time), monitor the execution until complete:

In [29]:
import json
def get_request(process_id, start_datetime, end_datetime, geometry):
    return"""
<?xml version="1.0" encoding="UTF-8"?>
<wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd">
  <ows:Identifier>%s</ows:Identifier>
  <wps:DataInputs>
    <wps:Input>
      <ows:Identifier>start_datetime</ows:Identifier>
      <wps:Data>
        <wps:ComplexData>{"type":"object","properties":{"timestamp":{"type":"string","format":"date-time","date-time":"%s"}}}</wps:ComplexData>
      </wps:Data>
    </wps:Input>
    <wps:Input>
      <ows:Identifier>end_datetime</ows:Identifier>
      <wps:Data>
        <wps:ComplexData>{"type":"object","properties":{"timestamp":{"type":"string","format":"date-time","date-time":"%s"}}}</wps:ComplexData>
      </wps:Data>
    </wps:Input>
    <wps:Input>
      <ows:Identifier>geometry</ows:Identifier>
      <wps:Data>
        <wps:ComplexData>{"type":"FeatureCollection","features":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[%s]]}}]}</wps:ComplexData>
      </wps:Data>
    </wps:Input>
  </wps:DataInputs>
  <wps:ResponseForm>
    <wps:ResponseDocument storeExecuteResponse="true" status="true"/>
  </wps:ResponseForm>
</wps:Execute>
""" % (process_id, start_datetime, end_datetime, geometry)

req = get_request("LS8 NBART Polygon Drill Raw Bands", "2016-02-15T00:00", "2016-10-22T00:00", "[114.4,-28.1],[114.8,-28.13],[114.8,-27.94],[114.4,-27.94],[114.4,-28.12]")

execution = wps.execute(None, None, request=req, mode='sync', output=[('output', True)])
outputRaw = execution.processOutputs[0].data[0]
output = json.loads(outputRaw)

# output.data is the csv payload returned from the polygon drill WPS
print(output['data'])


date,Red,Nir
2016-02-15T02:16:33.000Z,1887.620359,2746.580242
2016-02-15T02:16:57.000Z,1906.003066,2775.218789
2016-02-22T02:22:42.000Z,,
2016-02-22T02:23:05.000Z,,
2016-02-24T02:10:20.000Z,,
2016-02-24T02:10:44.000Z,,
2016-03-02T02:16:31.000Z,1937.491677,2795.786958
2016-03-02T02:16:55.000Z,1917.746013,2800.477701
2016-03-09T02:22:40.000Z,,
2016-03-09T02:23:04.000Z,,
2016-03-11T02:10:18.000Z,,
2016-03-11T02:10:42.000Z,,
2016-03-18T02:16:26.000Z,2009.859653,2866.939242
2016-03-18T02:16:49.000Z,1989.427765,2865.954440
2016-03-25T02:22:32.000Z,,
2016-03-25T02:22:56.000Z,,
2016-03-27T02:10:09.000Z,,
2016-03-27T02:10:33.000Z,,
2016-04-03T02:16:16.000Z,1871.431702,2669.126961
2016-04-03T02:16:40.000Z,1878.737229,2700.615954
2016-04-10T02:22:25.000Z,,
2016-04-10T02:22:49.000Z,,
2016-04-12T02:10:03.000Z,,
2016-04-12T02:10:27.000Z,,
2016-04-19T02:16:10.000Z,1986.719429,2766.400233
2016-04-19T02:16:34.000Z,2087.038900,2903.084727
2016-04-26T02:22:19.000Z,,
2016-04-26T02:22:43.000Z,,
2016-04-28T

This can also be done using the command line via POST:

In [15]:
import subprocess

subprocess.call(['curl', '-X', 'POST', '-d', '@payload.xml', '-o', 'result_22nov.json', 'http://130.56.242.16/ows/internal/wps?service=WPS'])

0

### Plotting output of WPS request

In [17]:
# Import libraries

import csv
import numpy as np
import datetime as dt
import matplotlib.dates as mdates

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file


In [18]:
# extract the data we want to plot

results = []
with open('result_long.txt', newline='') as inputfile:
    for row in csv.reader(inputfile):
        results.append(row)

orig_dates = results[0::5]
orig_PV = results[1::5]
orig_NPV = results[2::5]
orig_BS = results[3::5]
orig_Total = results[4::5]

dates_array = np.asarray([item for sublist in orig_dates for item in sublist])
PV_array = np.asarray([item for sublist in orig_PV for item in sublist])
NPV_array = np.asarray([item for sublist in orig_NPV for item in sublist])
BS_array = np.asarray([item for sublist in orig_BS for item in sublist])
Total_array = np.asarray([item for sublist in orig_Total for item in sublist])

PV = PV_array.astype(np.float)
NPV = NPV_array.astype(np.float)
BS = BS_array.astype(np.float)
Total = Total_array.astype(np.float)

# make dates python plottable
dt_dates = [dt.datetime.strptime(date,'%Y-%m-%dT%H:%M:%S.000Z').date() for date in dates_array ]
dates = mdates.date2num(list(dt_dates))

**Plotting our time-series using Bokeh** 

In [19]:
######################################################################################################
#                                                                                                    #
#   Bokeh is an interactive visualization library that targets modern web browsers for presentation. #
#   Its goal is to provide elegant, concise construction of versatile graphics, and to extend this   #
#   capability with high-performance interactivity over very large or streaming datasets.            #
#                                                                                                    #
#   For more information about Bokeh, please visit https://bokeh.pydata.org/en/latest/               #
#                                                                                                    #
######################################################################################################

## Plot PV NPV BS Total all on the same graph

p = figure(x_axis_type="datetime", title="8-day Fractional Cover Drill", plot_height=800, plot_width=1600)
p.xgrid.grid_line_color=None
p.background_fill_color = "beige"
p.background_fill_alpha = 0.5
p.ygrid.grid_line_alpha=0.5
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = '%'

p.line(dt_dates, PV, color = 'green', legend='PV')
p.line(dt_dates, NPV, color = 'blue', legend='NPV')
p.line(dt_dates, BS, color = 'red', legend='BS')
p.line(dt_dates, Total, color = 'orange', legend='Total')

show(p)

In [20]:
## Plot seperate graphs for each of PV NPV BS Total

from bokeh.plotting import figure, show
from bokeh.palettes import Spectral6
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid, Legend, LegendItem
from bokeh.layouts import row, column, gridplot

p1 = figure(x_axis_type="datetime", title="PV", plot_height=350, plot_width=800)
p1.xgrid.grid_line_color=None
#p1.background_fill_color = "beige"
#p1.background_fill_alpha = 0.5
p1.ygrid.grid_line_alpha=0.5
p1.xaxis.axis_label = 'Time'
p1.yaxis.axis_label = '%'
p1.line(dt_dates, PV, color = 'green')#, legend='PV')

p2 = figure(x_axis_type="datetime", title="NPV", plot_height=350, plot_width=800)
p2.xgrid.grid_line_color=None
#p2.background_fill_color = "beige"
#p2.background_fill_alpha = 0.5
p2.ygrid.grid_line_alpha=0.5
p2.xaxis.axis_label = 'Time'
p2.yaxis.axis_label = '%'
p2.line(dt_dates, NPV, color = 'blue')#, legend='NPV')

p3 = figure(x_axis_type="datetime", title="BS", plot_height=350, plot_width=800)
p3.xgrid.grid_line_color=None
p3.ygrid.grid_line_alpha=0.5
p3.xaxis.axis_label = 'Time'
p3.yaxis.axis_label = '%'
p3.line(dt_dates, BS, color = 'red')#, legend='BS')

p4 = figure(x_axis_type="datetime", title="Total", plot_height=350, plot_width=800)
p4.xgrid.grid_line_color=None
p4.ygrid.grid_line_alpha=0.5
p4.xaxis.axis_label = 'Time'
p4.yaxis.axis_label = '%'
p4.line(dt_dates, Total, color = 'orange')#, legend='Total')

p = gridplot([[p1,p2],[p3,p4]])

show(p)
